In [14]:
from webapp import create_app
from vk_parser import *
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
import joblib
from time import time
%matplotlib inline

# Обработка текста.

- Обработка текста через регулярные выражения.
- Удаление всех url из текста.
- Удаление хэштегов, если они есть в тексте.
- Удаление имён пользователей.
- Удаление пунктуации и цифр.
- Приведение всего текста к нижнему регистру.

In [4]:
def preprocess_text(text):
    text = text.lower().replace('ё', 'е')
    text = text.replace('d', '')
    text = text.replace('rt', '')
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'\b[a-zA-Zа-яА-Я1-9]\b', '', text)
    text = re.sub(r'\b(?:[аевзпхщчоумы][аиеоузпхщчмы]+)+\b', '', text)
    text = re.sub(r'[0-9]', '', text)    
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

# Считаем частоту слов в нашем предобработанном датасете

In [7]:
wordfreq = {}

for token in default_list:
    if token not in wordfreq.keys():
        wordfreq[token] = 1
    else:
        wordfreq[token] += 1

NameError: name 'default_list' is not defined

In [19]:
res = np.array(default_list)

In [20]:
unique_res = np.unique(res) 
unique_res

array(['ааа', 'ааааа', 'абаддон', ..., 'ярость', 'ясно', 'ёлочка'],
      dtype='<U19')

In [21]:
len(unique_res)

3507

In [22]:
bd_unique_words = unique_res.tolist()

# Предобработка текста.

- Используем корпус Юлии Рубцовой русских твитов для тренировки модели.
- Скачиваем корпус позитивных и негативных твитов с сайта http://study.mokoron.com/
- Добавляем дополнительную колонку для позитивных (1) и негативных твитов (0), тем самым размечая данные.

In [2]:
# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('training_data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('training_data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])

raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [5]:
app = create_app()
with app.app_context():
    query = Comment.query.all()
    preprop_comm = []

    for comment in query:
        preprop_comm.append(preprocess_text(comment.comment_text))

In [6]:
data = [preprocess_text(t) for t in raw_data]

In [7]:
bd_data = [preprocess_text(t) for t in preprop_comm]

In [8]:
print(len(labels), len(raw_data), len(data))

223846 223846 223846


In [9]:
print(len(bd_data))

1220


- Скачиваем набор русских стоп-слов для предобработки наших твитов.

In [10]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

- Разбиваем предобработанные слова на токены.
- Удаляем стоп слова.
- Производим лемматизацию или стемминг текста.
- Удаляем короткие слова длинной меньше 2 символов.

In [11]:
def tokenaizer(data):
    tokens= [word_tokenize(sent) for sent in data]
    return tokens

In [12]:
tokenized_data = tokenaizer(data)

In [13]:
print(len(tokenized_data))

223846


In [34]:
bd_tokenized_data = tokenaizer(bd_data)

In [35]:
print(len(bd_tokenized_data))

1220


# Стемминг слов

In [15]:
def stemmer(word, stem_init):
    return stem_init.stem(str(word))

In [16]:
def delete_stop_words_with_stemming(words, stop_words):    
    default_list = []
    for word in words:
        if word not in stop_words:
            stemmed_word = stemmer(word, snowball)
            default_list.append(stemmed_word)

    return list(default_list)

In [17]:
stop_words = set(stopwords.words("russian"))
snowball = SnowballStemmer(language="russian")
stemmed_words = []
counter = 0
for part_of_list in tokenized_data:
    stemmed_words.append(delete_stop_words_with_stemming(part_of_list, stop_words))
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 5
Processed 3000 of 15
Processed 4000 of 8
Processed 5000 of 13
Processed 6000 of 13
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 18
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 5
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 9
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 10
Processed 22000 of 2
Processed 23000 of 6
Processed 24000 of 11
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 4
Processed 29000 of 13
Processed 30000 of 9
Processed 31000 of 14
Processed 32000 of 3
Processed 33000 of 4
Processed 34000 of 6
Processed 35000 of 16
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 15
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 5
Processed 45000 of 9
Processed 46000 of 12
Processed 47000 of 9
Proc

In [20]:
print(len(stemmed_words))

223846


# Лемматизация слов.

"Экземпляры класса MorphAnalyzer обычно занимают порядка 15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь организовать свой код так, чтобы создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем."

https://pymorphy2.readthedocs.io/en/latest/user/guide.html

In [17]:
def delete_stop_words_with_lemmatizing(words, morph, stop_words):  
    default_list = []
    for word in words:
        if word not in stop_words:
            lem_word = morph.parse(str(word))[0].normal_form
            default_list.append(lem_word)
    return list(default_list)

In [27]:
stop_words = set(stopwords.words("russian"))
morph = pymorphy2.MorphAnalyzer()
lemmatized_words = []
counter = 0
for part_of_list in tokenized_data:
    lemmatized_words.append(delete_stop_words_with_lemmatizing(part_of_list, morph, stop_words))   
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 5
Processed 3000 of 15
Processed 4000 of 8
Processed 5000 of 13
Processed 6000 of 13
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 18
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 5
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 9
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 10
Processed 22000 of 2
Processed 23000 of 6
Processed 24000 of 11
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 4
Processed 29000 of 13
Processed 30000 of 9
Processed 31000 of 14
Processed 32000 of 3
Processed 33000 of 4
Processed 34000 of 6
Processed 35000 of 16
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 15
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 5
Processed 45000 of 9
Processed 46000 of 12
Processed 47000 of 9
Proc

In [28]:
len(lemmatized_words)

223846

In [36]:
bd_lemmatized_words = []
counter = 0
for part_of_list in bd_tokenized_data:
    bd_lemmatized_words.append(delete_stop_words_with_lemmatizing(part_of_list, morph, stop_words))   
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 2


In [37]:
len(bd_lemmatized_words)

1220

In [38]:
bd_lemmatized_words = [x for x in bd_lemmatized_words if x != []] #удалил пустые списки после лемматизации

In [39]:
len(bd_lemmatized_words)

1070

In [31]:
def to_data_frame(data, labels):
    default_list = []
    
    
    for string in data:
        new_string = ' '.join(string)
        default_list.append(new_string)
    
    
    default_list = pd.DataFrame(list(zip(default_list, labels)), columns =['comments', 'sentiment'])
    return default_list

In [32]:
df_stemmed_words = to_data_frame(stemmed_words, labels)

NameError: name 'stemmed_words' is not defined

In [33]:
df_lemmatized_words = to_data_frame(lemmatized_words, labels)

In [40]:
df_lemmatized_words.head()

,comments,sentiment
0,школотый поверь самый общество профилировать п...,1
1,таки немного похожий мальчик равно,1
2,идиотка испугаться,1
3,угол сидеть погибать голод порция взять хотя ж...,1
4,значит страшилка блин посмотреть часть создать...,1


- сохраним наш датасет

In [34]:
df_lemmatized_words.to_csv('lemmatized_words.csv', index=False, encoding='cp1251')

In [44]:
df_stemmed_words.to_csv('stemmed_words.csv', index=False, encoding='cp1251')

In [39]:
df_bd_lemmatized_words.to_csv('X_test.csv', index=False, encoding='cp1251')

- загрузим наш датасет

In [19]:
df_lemmatized_words = pd.read_csv('lemmatized_words.csv', encoding='cp1251')

In [20]:
df_stemmed_words = pd.read_csv('stemmed_words.csv', encoding='cp1251')

In [22]:
df_bd_lemmatized_words = pd.read_csv('X_test.csv', encoding='cp1251')

In [41]:
df_bd_lemmatized_words.head()

,comments
0,толсто аж тонко весело
1,казаться это пародия мойти пайтон
2,типичный осада железный войнов крепость имперс...
3,просто немного унижение железный плакса
4,удивлённый


# Word embedding (Векторное представление слов)

- Сделаем векторизацию текста 4-мя способами BoW, TF-IDF, Word2Vec, doc2vec.
- На этих данных обучим модели и сравним результаты.

# Bag Of Words (BoW).

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_lemmatized_words['comments'].values.astype('U'))

In [62]:
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

In [51]:
vectorizer = joblib.load('vectorizer.pkl')
X = vectorizer.fit_transform(df_lemmatized_words['comments'].values.astype('U'))

In [63]:
print(vectorizer.get_feature_names()[:100])

['nan', 'аа', 'ааа', 'аааааааааать', 'ааааааааать', 'аааааааад', 'ааааааааррррра', 'ааааааий', 'ааааааррррра', 'ааааадееть', 'ааааайщий', 'аааааллиловать', 'ааааапасность', 'ааааатыдыщ', 'ааааать', 'ааааахуенноооо', 'ааааахууууууеееееть', 'аааад', 'аааалиловать', 'ааааня', 'аааас', 'аааать', 'аааафигеть', 'аааахааэаэхахаэахах', 'аааахиренный', 'аааахууууууууууууенно', 'аааввва', 'ааад', 'ааадайтесилпопростиьунегопрощение', 'аааж', 'ааанечка', 'аааня', 'ааар', 'ааарг', 'ааармотавва', 'ааарра', 'ааастыыыыыый', 'ааать', 'ааахерить', 'аав', 'ааваас', 'ааввввва', 'аавдаа', 'аагаад', 'аагрх', 'аад', 'аадский', 'аажмаар', 'аай', 'аакаунт', 'аалга', 'ааллар', 'аалтонить', 'аан', 'аанга', 'аанхаа', 'ааплодисменты', 'аар', 'аарон', 'аарррра', 'ааррррра', 'аархаж', 'аарцна', 'аарцтая', 'аас', 'аата', 'аатаня', 'ааура', 'аахахахтак', 'ааххахазывъв', 'аашилдаг', 'аашлаад', 'аащщащпщпщащпщашуоцлвзццдьать', 'ааьхахахахъаъахахъахаазаззазазазазазмжааа', 'ааэаэахахахахах', 'аая', 'ааяаа', 'ааяая', 'ааяг

In [39]:
print(X.shape)

(223846, 94121)


# Bag Of Word with n-grams.

In [58]:
vectorizer2 = CountVectorizer(ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(df_lemmatized_words['comments'].values.astype('U'))

In [64]:
joblib.dump(vectorizer2, 'vectorizer2.pkl')

['vectorizer2.pkl']

In [65]:
vectorizer2 = joblib.load('vectorizer2.pkl')
X2 = vectorizer2.fit_transform(df_lemmatized_words['comments'].values.astype('U'))

In [66]:
print(vectorizer2.get_feature_names()[:100])

['аа акаарыбын', 'аа ынырык', 'ааа давно', 'аааааааааать выкладывать', 'ааааааааать нравиться', 'аааааааад губерний', 'ааааааааррррра малый', 'ааааааий вспомнить', 'ааааадееть мозг', 'ааааайщий какой', 'аааааллиловать красотка', 'ааааапасность любить', 'ааааатыдыщ ого', 'ааааать блииина', 'ааааня хотеть', 'аааас болеть', 'аааать жить', 'аааать солнышко', 'аааать спасибо', 'аааать яй', 'аааафигеть закончить', 'аааафигеть просто', 'аааахиренный картинка', 'аааввва книжный', 'аааж противно', 'аааня приходить', 'ааар быстрый', 'ааармотавва точно', 'ааарра нравиться', 'ааать больно', 'ааать коллекция', 'ааать любить', 'ааать удача', 'ааахерить особенно', 'аав гийна', 'аав мааня', 'аав нь', 'аав таныга', 'аав чухать', 'аав ээж', 'аав ээжийна', 'аав ээжтэйгээ', 'ааваас авахыга', 'ааввввва родди', 'аавдаа зодуулсан', 'аавдаа тас', 'аагаад бхад', 'аагрх мазафак', 'аад весь', 'аад оны', 'аад оныхон', 'аадский боль', 'аажмаар алга', 'аай бог', 'аай зря', 'аай каждый', 'аай решить', 'аай скучно', 

In [67]:
print(X2.shape)

(223846, 754286)


- Вывод: 
  - Потеря порядка слов;
  - Полученные вектора, не являются информативными и занимают очень много места в памяти;

# TF-IDF.

(TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса.

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(data,tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

In [116]:
tfidf_vect = TfidfVectorizer()
#tfidf_vect_fit = tfidf_vect.fit(X_train)
tfidf_vect_train = tfidf_vect.fit_transform(df_lemmatized_words['comments'].values.astype('U'))
#X_train = vectorize(X_train,tfidf_vect_fit)

In [117]:
tfidf_vect_test = tfidf_vect.transform(df_bd_lemmatized_words['comments'].values.astype('U'))

In [132]:
tfidf_vect_train.shape

(223846, 91962)

In [133]:
tfidf_vect_test.shape

(1070, 91962)

# Word2Vec для предложений.

In [79]:
from gensim.models import Word2Vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [82]:
num_features = 1000  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

In [83]:
df_lemmatized_words['comments'].head()

0    школотый поверь самый общество профилировать п...
1                   таки немного похожий мальчик равно
2                                   идиотка испугаться
3    угол сидеть погибать голод порция взять хотя ж...
4    значит страшилка блин посмотреть часть создать...
Name: comments, dtype: object

In [84]:
print("Training model....")
model = Word2Vec(lemmatized_words, #пихаем сюда список списков наших слов
                workers=num_workers,
                size=num_features,
                min_count=min_word_count,
                window=context,
                sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "1000features_40minwords_10context"
model.save(model_name)

2021-01-19 21:35:54,215 : INFO : collecting all words and their counts
2021-01-19 21:35:54,216 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-19 21:35:54,238 : INFO : PROGRESS: at sentence #10000, processed 65179 words, keeping 14918 word types
2021-01-19 21:35:54,256 : INFO : PROGRESS: at sentence #20000, processed 128674 words, keeping 22840 word types
2021-01-19 21:35:54,281 : INFO : PROGRESS: at sentence #30000, processed 194264 words, keeping 29236 word types
2021-01-19 21:35:54,302 : INFO : PROGRESS: at sentence #40000, processed 258322 words, keeping 34598 word types
2021-01-19 21:35:54,323 : INFO : PROGRESS: at sentence #50000, processed 322449 words, keeping 39565 word types
2021-01-19 21:35:54,348 : INFO : PROGRESS: at sentence #60000, processed 386089 words, keeping 44137 word types
2021-01-19 21:35:54,371 : INFO : PROGRESS: at sentence #70000, processed 450981 words, keeping 48675 word types
2021-01-19 21:35:54,391 : INFO : PROGRESS: at s

Training model....


2021-01-19 21:35:54,434 : INFO : PROGRESS: at sentence #100000, processed 643840 words, keeping 59888 word types
2021-01-19 21:35:54,460 : INFO : PROGRESS: at sentence #110000, processed 707136 words, keeping 63221 word types
2021-01-19 21:35:54,481 : INFO : PROGRESS: at sentence #120000, processed 768653 words, keeping 66362 word types
2021-01-19 21:35:54,505 : INFO : PROGRESS: at sentence #130000, processed 829112 words, keeping 69081 word types
2021-01-19 21:35:54,528 : INFO : PROGRESS: at sentence #140000, processed 890601 words, keeping 71841 word types
2021-01-19 21:35:54,550 : INFO : PROGRESS: at sentence #150000, processed 951762 words, keeping 74459 word types
2021-01-19 21:35:54,570 : INFO : PROGRESS: at sentence #160000, processed 1013405 words, keeping 76940 word types
2021-01-19 21:35:54,593 : INFO : PROGRESS: at sentence #170000, processed 1074752 words, keeping 79397 word types
2021-01-19 21:35:54,615 : INFO : PROGRESS: at sentence #180000, processed 1135477 words, keepi

In [131]:
model.wv.doesnt_match("мужик король парень дерево".split())

C:\Users\kamikaze666\Anaconda3\envs\tf_gpu\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'король'

In [130]:
model.wv.most_similar("парень")

2021-01-19 23:24:37,856 : INFO : precomputing L2-norms of word weight vectors


[('чёлочка', 0.8661671280860901),
 ('котов', 0.8649200201034546),
 ('акцент', 0.8526308536529541),
 ('красавица', 0.8497965335845947),
 ('чакра', 0.8473725318908691),
 ('серебрянка', 0.8470666408538818),
 ('магаза', 0.8436814546585083),
 ('челюсть', 0.8363137245178223),
 ('киснуть', 0.83525550365448),
 ('плагин', 0.8299589157104492)]

In [88]:
model.wv.vectors.shape

(4204, 1000)

- Функция для усреднения всех векторов слов в абзаце.

In [89]:
def featureVecMethod(words, model, num_features):
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model.wv[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec

- Функция вычисления среднего вектора признаков.

In [90]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [91]:
trainDataVecs = getAvgFeatureVecs(lemmatized_words, model, num_features)

Review 0 of 223846
Review 1000 of 223846


<ipython-input-89-907d822aa921>:11: RuntimeWarning: invalid value encountered in true_divide
  featureVec = np.divide(featureVec, nwords)


Review 2000 of 223846
Review 3000 of 223846
Review 4000 of 223846
Review 5000 of 223846
Review 6000 of 223846
Review 7000 of 223846
Review 8000 of 223846
Review 9000 of 223846
Review 10000 of 223846
Review 11000 of 223846
Review 12000 of 223846
Review 13000 of 223846
Review 14000 of 223846
Review 15000 of 223846
Review 16000 of 223846
Review 17000 of 223846
Review 18000 of 223846
Review 19000 of 223846
Review 20000 of 223846
Review 21000 of 223846
Review 22000 of 223846
Review 23000 of 223846
Review 24000 of 223846
Review 25000 of 223846
Review 26000 of 223846
Review 27000 of 223846
Review 28000 of 223846
Review 29000 of 223846
Review 30000 of 223846
Review 31000 of 223846
Review 32000 of 223846
Review 33000 of 223846
Review 34000 of 223846
Review 35000 of 223846
Review 36000 of 223846
Review 37000 of 223846
Review 38000 of 223846
Review 39000 of 223846
Review 40000 of 223846
Review 41000 of 223846
Review 42000 of 223846
Review 43000 of 223846
Review 44000 of 223846
Review 45000 of 223

In [135]:
trainDataVecs.shape

(223846, 1000)

In [123]:
testDataVecs = getAvgFeatureVecs(bd_lemmatized_words, model, num_features)

Review 0 of 1070


ValueError: operands could not be broadcast together with shapes (1000,) (20,) 

In [124]:
testDataVecs.shape

(1070, 1000)

# Doc2Vec.

In [100]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(d, tags=[str(i)]) for i, d in enumerate(lemmatized_words)]

In [102]:
max_epochs = 20
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                workers=4)
  
model.build_vocab(tagged_data)

2021-01-19 21:41:03,187 : INFO : collecting all words and their counts
2021-01-19 21:41:03,188 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-01-19 21:41:03,226 : INFO : PROGRESS: at example #10000, processed 65179 words (1761570/s), 14918 word types, 10000 tags
2021-01-19 21:41:03,267 : INFO : PROGRESS: at example #20000, processed 128674 words (1614851/s), 22840 word types, 20000 tags
2021-01-19 21:41:03,311 : INFO : PROGRESS: at example #30000, processed 194264 words (1512694/s), 29236 word types, 30000 tags
2021-01-19 21:41:03,353 : INFO : PROGRESS: at example #40000, processed 258322 words (1564156/s), 34598 word types, 40000 tags
2021-01-19 21:41:03,393 : INFO : PROGRESS: at example #50000, processed 322449 words (1606383/s), 39565 word types, 50000 tags
2021-01-19 21:41:03,429 : INFO : PROGRESS: at example #60000, processed 386089 words (1804888/s), 44137 word types, 60000 tags
2021-01-19 21:41:03,478 : INFO : PROGRESS: at example #70000, pr

In [50]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
Model Saved


In [103]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("я люблю картошку".lower())
v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
#print(similar_doc)

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.docvecs['1'])

2021-01-19 21:42:24,189 : INFO : loading Doc2Vec object from d2v.model
2021-01-19 21:42:25,183 : INFO : loading vocabulary recursively from d2v.model.vocabulary.* with mmap=None
2021-01-19 21:42:25,184 : INFO : loading trainables recursively from d2v.model.trainables.* with mmap=None
2021-01-19 21:42:25,185 : INFO : loading wv recursively from d2v.model.wv.* with mmap=None
2021-01-19 21:42:25,185 : INFO : loading docvecs recursively from d2v.model.docvecs.* with mmap=None
2021-01-19 21:42:25,186 : INFO : loaded d2v.model
2021-01-19 21:42:25,406 : INFO : precomputing L2-norms of doc weight vectors


# Подготовка лемматизированных данных для обучения, валидации и тестирования.

In [129]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_words['comments'],lemmatized_words['sentiment'],test_size = 0.90, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

print("Data distribution:\n- Train: {} \n- Validation: {} \n- Test: {}".format(len(y_train),len(y_val),len(y_test)))

Data distribution:
- Train: 11192 
- Validation: 5596 
- Test: 5596


In [37]:
default_list = []
for string in bd_lemmatized_words:
    new_string = ' '.join(string)
    default_list.append(new_string)

In [38]:
bd_lemmatized_words = pd.DataFrame(list(default_list), columns =['comments']) 

In [126]:
bd_lemmatized_words['sentiment'] = np.nan

TypeError: list indices must be integers or slices, not str

In [127]:
X_test = bd_lemmatized_words['comments']

TypeError: list indices must be integers or slices, not str

In [128]:
y_train = lemmatized_words['sentiment']

TypeError: list indices must be integers or slices, not str

In [24]:
X_test.head()

0                               толсто аж тонко весело
1                    казаться это пародия мойти пайтон
2    типичный осада железный войнов крепость имперс...
3              просто немного унижение железный плакса
4                                           удивлённый
Name: comments, dtype: object

In [18]:
X_train.head()

172855    завтра ужасно пойти школа пойти ладный выспать...
23908            выражение услышать паршивый овца бочка мёд
172208                        задержка вылет час нафиг жить
162818                                бля вообще лень учить
33933                            помнить лето рука валанчик
Name: comments, dtype: object

In [20]:
y_test.head()

110494    1
143489    0
174023    0
120179    0
165229    0
Name: sentiment, dtype: int64

# Random Forest Classifier на Word2Vec.

In [109]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators = 50,
                                oob_score=True,
                                random_state=42,
                                verbose=2)
    
print("Fitting random forest to training data....")    
forest = forest.fit(np.nan_to_num(trainDataVecs), df_lemmatized_words["sentiment"])

Fitting random forest to training data....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.2s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 10.5min finished


In [110]:
# Predicting the sentiment values for test data and saving the results in a csv file 
result = forest.predict(np.nan_to_num(testDataVecs))
output = pd.DataFrame(data={"comments":bd_lemmatized_words, "sentiment":result})

accuracy = accuracy_score(output['sentiment'], result)
print(f'Out-of-bag score estimate: {forest.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

Out-of-bag score estimate: 0.645
Mean accuracy score: 1.0


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [138]:
w2v_scores = cross_val_score(forest, np.nan_to_num(trainDataVecs), df_lemmatized_words["sentiment"], cv=5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.6s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  8.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  7.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.6s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  7.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.4s finished


In [139]:
print(w2v_scores)
w2v_scores.mean()

[0.63984811 0.64890884 0.63528334 0.62637093 0.63036923]


0.6361560908302059

In [112]:
output.to_csv("output.csv", index=False, encoding='cp1251')

In [113]:
output.head(20)

,comments,sentiment
0,"[толсто, аж, тонко, весело]",0
1,"[казаться, это, пародия, мойти, пайтон]",1
2,"[типичный, осада, железный, войнов, крепость, ...",1
3,"[просто, немного, унижение, железный, плакса]",1
4,[удивлённый],0
5,"[кто, интересно, читать, классно, осада, жв, о...",1
6,[перевод],1
7,"[юмор, весь, полок, смеяться]",1
8,[ересь],1
9,"[госпожа, комиссар, бдеть, дисциплина]",1


# Random Forest Classifier на TF-IDF.

- Мы исследуем первые результаты классификатора Random Forest без какой-либо настройки гиперпараметров с использованием 5-кратной проверки.

In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
RFC_clf = RandomForestClassifier(verbose=2)
scores = cross_val_score(RFC_clf, X_train, y_train.values.ravel(), cv=5)

In [43]:
print(scores)
scores.mean()

[0.63912461 0.63867798 0.64164433 0.62689902 0.62555853]


0.6343808934235531

In [203]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [45]:
from sklearn.model_selection import GridSearchCV

RF_clf = RandomForestClassifier(verbose=2)
parameters = {
    'n_estimators': [5,50,100],
    'max_depth': [2,10,20,None]
}

cv = GridSearchCV(RF_clf, parameters)
cv.fit(X_train,y_train.values.ravel())
print_results(cv)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


building tree 1 of 5
building tree 2 of 5
building tree 3 of 5
building tree 4 of 5
building tree 5 of 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


building tree 1 of 5
building tree 2 of 5
building tree 3 of 5
building tree 4 of 5
building tree 5 of 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  7.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.8s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 11.3min finished


# Decision Tree Classifier на TF-IDF.

In [4]:
from sklearn import tree

In [70]:
DTC_clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=200)

In [71]:
DTC_clf.fit(X_train, y_train.values.ravel())

DecisionTreeClassifier(criterion='entropy', max_depth=200)

In [72]:
score = DTC_clf.score(X_train, y_train.values.ravel())

In [73]:
score

0.7706397426733381

In [74]:
pred = DTC_clf.predict(X_test)

In [76]:
y_test = pd.DataFrame(pred, columns = ['sentiment'])

In [77]:
y_test.to_csv('y_test.csv', index=False, encoding='cp1251')

In [ ]:
result = pd.concat([bd_lemmatized_words, df], axis=1)

# Evaluating the models.

In [ ]:
vectorizers = [CountVectorizer, TFIDFVEctorizer, word2vec_vectorizer, doc2vec_vectorizer]
models = [RandomForestClassifier, ...]

def preprocess(initial_dataset): ...

    clean_posts = preprocess(initial_dataset)
    stemmed_words = ...
    lemmatized_words = ...

    param_grid = {RandomForestClassifier: RFC_grid} #наборы пааметров для данной модели
    # ключем может быть класс из sklearn а значением словарь параметров



normalized_corpuses = [stemmed_words, lemmatized_words]

def evaluate_model(model, X):
    evaluated_model = {"model": model} #значением может быть класс из sklearn
    # Создаем и фитим RandomSearchCV с учетом param_grid
    #добавляем best params  
    evaluated_model["best_param"] = best_params
    # Создаем модель с лучшими параметрами
    # обучаем моедль на X_train y_train
    # предсказываем на X_test
    # считаем метрики на предсказанных результатах и y_test
    evaluated_model["metric"] = metric
    # лучший результат добавляем в result

def evaluate_models(normalized_corpuses, vectorizers, models):
    evaluated_models = []
    for normalized_corpus in normalized_corpuses:
        X_train, X_test, y_train, y_test = train_test_split(...)
        for vectorizer in vectorizers:
            #  в идеале тут было бы просто vectorized_data = vectorizer.fit_transform(X_train)
            # но так как сейчас нету возможности так сделать для word2vec и doc2vec
            # то тут будет кастомная функция, например word2vec_vectorizer
            # которая будет делать преобазование в вектор по методу word2vec
            # и тут будет примерно следующее
            if vectorizer == word2vec_vectorizer:
                # получаем X_train, X_test, y_train, y_test через word2vec 
                # X_test и y_test на основе normalized_corpus скачанные данные не берем пока

            elif vectorizer == doc2vec_vectorizer:
                # получаем X_train, X_test, y_train, y_test через doc2vec 
            elif vectorizer in (CountVectorizer, TFIDFVEctorizer):
                # получаем X_train, X_test, y_train, y_test через соответствующий векторайзер 
            
            for model in models:
                evaluated_model = evaluate_model(model, X_train, X_test, y_train, y_test)
                evaluated_models.append(evaluated_model)
    return evaluated_models